# Audio I/O Module Implementation

This notebook implements the Audio I/O Module for PolyLingua, including the `AudioFormat` type and `AudioProcessor` class with all required methods.

In [1]:
# Import necessary libraries
from dataclasses import dataclass
from typing import Optional
import io
import numpy as np
from pydub import AudioSegment
import soundfile as sf

In [2]:
# Define AudioFormat type
@dataclass
class AudioFormat:
    sample_rate: int
    channels: int
    encoding: str  # e.g., 'wav', 'mp3', etc.

In [3]:
# Define AudioProcessor class
class AudioProcessor:
    def __init__(self):
        self.audio: Optional[AudioSegment] = None
        self.format: Optional[AudioFormat] = None
    
    def load_audio_from_bytes(self, audio_bytes: bytes, input_format: AudioFormat) -> None:
        """Load audio from bytes using pydub."""
        self.audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format=input_format.encoding)
        self.format = input_format
    
    def resample(self, target_sample_rate: int) -> None:
        """Resample audio to target sample rate."""
        if self.audio:
            self.audio = self.audio.set_frame_rate(target_sample_rate)
            self.format.sample_rate = target_sample_rate
    
    def convert_to_mono(self) -> None:
        """Convert audio to mono."""
        if self.audio and self.audio.channels > 1:
            self.audio = self.audio.set_channels(1)
            self.format.channels = 1
    
    def normalize_volume(self, target_dbfs: float = -20.0) -> None:
        """Normalize audio volume to target dBFS."""
        if self.audio:
            difference = target_dbfs - self.audio.dBFS
            self.audio = self.audio.apply_gain(difference)
    
    def export_to_wav_bytes(self) -> bytes:
        """Export audio to WAV bytes."""
        if self.audio:
            buffer = io.BytesIO()
            self.audio.export(buffer, format="wav")
            return buffer.getvalue()
        return b""
    
    def validate_audio_duration(self, max_seconds: float = 60.0) -> bool:
        """Validate audio duration."""
        if self.audio:
            duration = len(self.audio) / 1000.0  # pydub duration in ms
            return duration <= max_seconds
        return False

In [4]:
# Test the AudioProcessor class
processor = AudioProcessor()

# Example: Load a sample audio (assuming you have audio_bytes)
# For testing, you can use a dummy or load from file
# audio_bytes = b'...your audio data...'
# input_format = AudioFormat(sample_rate=44100, channels=2, encoding='wav')
# processor.load_audio_from_bytes(audio_bytes, input_format)

print("AudioProcessor class defined successfully.")

AudioProcessor class defined successfully.


In [5]:
# Test resample method
# processor.resample(16000)
# print(f"Resampled to {processor.format.sample_rate} Hz")

In [6]:
# Test convert_to_mono
# processor.convert_to_mono()
# print(f"Converted to {processor.format.channels} channel(s)")

In [7]:
# Test normalize_volume
# processor.normalize_volume()
# print(f"Normalized to {processor.audio.dBFS} dBFS")

In [8]:
# Test export_to_wav_bytes
# wav_bytes = processor.export_to_wav_bytes()
# print(f"Exported {len(wav_bytes)} bytes of WAV data")

In [9]:
# Test validate_audio_duration
# is_valid = processor.validate_audio_duration()
# print(f"Audio duration valid: {is_valid}")